<a href="https://colab.research.google.com/github/MatteoBettini/Stock-Market-Prediction-2020/blob/main/notebooks/FInal%20assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Take-home Assignment

# Imports

In [1]:
# To plot figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import sklearn
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from tensorflow import keras
assert tf.__version__ >= "2.0"

import os
import datetime
import pandas as pd
import numpy as np

# Load the TensorBoard notebook extension
%load_ext tensorboard

# To make this notebook's output stable across runs
random_state = 42
np.random.seed(random_state)
tf.random.set_seed(random_state)

plt.rcParams['figure.figsize'] = (15.0, 10.0)
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['figure.subplot.bottom'] = 0.125
plt.rcParams['figure.edgecolor'] = 'white'
plt.rcParams["savefig.dpi"] = 300

plt.rcParams['font.size'] = 15
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15

In [2]:

from google.colab import drive
drive.mount('/content/drive')
'''
ax = nasdaq_df.plot(y=["Close"])

fig = ax.get_figure()
fig.savefig('/content/drive/My Drive/Stock_market_figures/close.png')
'''

KeyboardInterrupt: ignored

# Dataset exploration

In this section we will upload and explore the dataset "**Processed_NASDAQ**",  containing several daily features of NASDAQ Composite from 2010 to 2017. The dataset was acquired from [this repository](https://archive.ics.uci.edu/ml/datasets/CNNpred%3A+CNN-based+stock+market+prediction+using+a+diverse+set+of+variables#).

It covers features from various categories of technical indicators, future contracts, price of commodities, important indices of markets around the world, price of major companies in the U.S. market, and treasury bill rates. Sources and thorough description of features have been mentioned in the paper "[CNNpred: CNN-based stock market prediction using a diverse set of variables](https://arxiv.org/pdf/1810.08923.pdf)".

## 1 - Loading the dataset

In [ ]:
nasdaq_url = 'https://raw.githubusercontent.com/MatteoBettini/Stock-Market-Prediction-2020/main/stock_markets_datasets/Processed_NASDAQ.csv?token=ANHXQQOLNU7CE7V2OTZJ43C76A552'
dji_url = 'https://raw.githubusercontent.com/MatteoBettini/Stock-Market-Prediction-2020/main/stock_markets_datasets/Processed_DJI.csv?token=ANHXQQOXYQZSSSTVFKX6RZS74XDXA'
nyse_url = 'https://raw.githubusercontent.com/MatteoBettini/Stock-Market-Prediction-2020/main/stock_markets_datasets/Processed_NYSE.csv?token=ANHXQQNAISMPCLVLRTGNJBC74XD2C'
russel_url = 'https://raw.githubusercontent.com/MatteoBettini/Stock-Market-Prediction-2020/main/stock_markets_datasets/Processed_RUSSELL.csv?token=ANHXQQPGLBLSM3B36OLWIPC74XD3U'
s_p_url = 'https://raw.githubusercontent.com/MatteoBettini/Stock-Market-Prediction-2020/main/stock_markets_datasets/Processed_S%26P.csv?token=ANHXQQNRFS3NKP2XCF5Q5MS74XD5K'

In [ ]:
nasdaq_df = pd.read_csv(nasdaq_url, parse_dates=["Date"], index_col="Date")
# Dataset is now stored in a Pandas Dataframe

We load the .csv file telling pandas to parse the date column and to use it to index the data.

## 2 - Exploring the dataset

Now that we have loaded the dataset we can start inspecting the data.

In [ ]:
nasdaq_df.head(10)

Taking a peak at the first ten elements we can already see that there are a lot of missing values. They will be treated accordingly in [this section](#missing_values).


We can also see that the dates inculuded in the dataset are referring only to working days as the stock market is open only on those days.

We can get a confirmation of this by looking at the following rows where we see that 16,17 January 2010 are not present because it was a weekend and 18 January 2010 is not present because of the federal U.S. festivity of "Martin Luther King Jr. Day".

This is not a problem for our machine learning pipeline as we will map the 'Date' feature into a categorical feature representing the day of the week.
More information on this will be provided in [this section](#cathegorical_values).

In [ ]:
nasdaq_df.iloc[9:13]

### Features

The datasets described in [the paper](https://arxiv.org/pdf/1810.08923.pdf) contain 1984 entries, each representing a day of trading in a stock market. Each entry has 82 features which are grouped in the following way:

*   Primitive features
*   Technical indicators
*   Economic data
*   World stock markets
*   The exchange rate of U.S. dollar
*   Commodities
*   Big U.S. Companies
*   Futures contracts

The authors have made available five datasets, each representing a different stock market. The available markets are: S&P 500, NASDAQ Composite, Dow Jones Industrial Average, RUSSELL 2000, and NYSE Composite. In this work we will explore and analyse the NASDAQ Composite dataset, but all the insights we are going to gain will be valid for all datasets.

The primitive featrues and the technical indicators are unique for each dataset, while all the other features are common among datasets.

A tabular description of the features is also reported in the following images.

<img src="https://raw.githubusercontent.com/MatteoBettini/Stock-Market-Prediction-2020/main/feature_description/feature_table_1.png?token=ANHXQQI7BINXDEPRGLZ42LS74XAH6" width="2000">
<img src="https://raw.githubusercontent.com/MatteoBettini/Stock-Market-Prediction-2020/main/feature_description/feature_table_2.png?token=ANHXQQOG2ZP4BPHXIWXNWWK74XAKY" width="2000">

Let's reorder the features in our data frame to match the description.

In [ ]:
technical_indeces = ["Volume","mom","mom1","mom2","mom3","ROC_5","ROC_10","ROC_15","ROC_20","EMA_10","EMA_20","EMA_50","EMA_200"]
economic_indices = ["DTB4WK","DTB3","DTB6","DGS5","DGS10","DAAA","DBAA","TE1","TE2","TE3","TE5","TE6","DE1","DE2","DE4","DE5","DE6","CTB3M","CTB6M","CTB1Y"]
comodities_indices = ["Oil","Brent","WIT-oil","Gold","gold-F","XAU","XAG","GAS-F","silver-F","copper-F","wheat-F"]
word_indices = ["GSPC","DJI","NYSE","RUT","HSI","SSEC","FCHI","FTSE","GDAXI"]
exchannge_indices = ["JPY","GBP","CAD","CNY","AUD","NZD","CHF","EUR","Dollar index","Dollar index-F"]
companies_indices = ["XOM","JPM","AAPL","MSFT","GE","JNJ","WFC","AMZN"]
futures_indices = ["CAC-F","FTSE-F","DAX-F","HSI-F","Nikkei-F","KOSPI-F","NASDAQ-F","DJI-F","S&P-F","RUSSELL-F"]


nasdaq_df = nasdaq_df[["Name","Close"] +
                       technical_indeces +
                       economic_indices +
                       comodities_indices +
                       word_indices +
                       exchannge_indices +
                       companies_indices +
                       futures_indices]

By looking at the info of the NASDAQ dataset we can see features, their types and the number of non-null values.

In [ ]:
nasdaq_df.info()

If we analyze carefully the features availaible in the actual dataset we may notice a few differences from those described in the paper.

Apart from the fact that the same feature may have different names in the two descriptions, in the loaded dataset we may find two **new** features:

| # | Feature | Description | Type |
| --- | --- | --- | --- |
| 83 | mom | Return of 1 day before | Technical indicator |
| 84| wheat-F | Relative change of wheat price| Comodity |

The features descriptions and types come from my best intution, as I could not find any detailed description ot these two features.

In each market's dataset we may also see that there is a feature indicating the name of the market from where the data comes.



In [ ]:
nasdaq_df["Name"].value_counts()

For now we are considering only one market so we can drop this feature.

In [ ]:
nasdaq_df.drop(labels=["Name"], axis=1, inplace=True)

<a name="return_equation_cell"/>

Lastly, **mom** represents the return of 1 day before at time $t$. So among the features IXIC, GSPC, DJI, NYSE and RUSSEL, the one referring to the market from where the dataset is from is dropped as the same data is contained in **mom**.
 
The return of a market (mom) at time $t$ can be computed from the "Close" feature as $$Return_{t} = \frac{Close_{t}}{Close_{t-1}} - 1$$

For example, in our dataset *Processed_NASDAQ* the feature IXIC is not present.

Therefore, we can understand why in our data frame we have 84 columns: Date (which is the index) and 83 features. This is the result of the addition of "mom", "wheat-F" and "Name" to the 82 features described in the paper considering the absence of one feature among IXIC, GSPC, DJI, NYSE and RUSSEL.

All features are floats except for Date (that has been parsed as datetime64) and Name that is a string.

---

We also want to insert the feature describing the day of the week.

We do this in the following way:

In [ ]:
nasdaq_df.insert(0,'day_of_week',nasdaq_df.index.dayofweek)

In [ ]:
nasdaq_df.head(3)

In [ ]:
nasdaq_df["day_of_week"].value_counts()

In [ ]:
nasdaq_df.describe()

This gives us some information on how the data are distributed. NaN values are excluded.

## 3 - Plotting the time series

The data in this dataset are representing a **time series**.

This is not the typical dataset for a supervised learning prediction, so we will have to modify some things in order to apply the supervised learning algorithms we know. This will be done in [this section](#transforming_the_dataset).

But first, let's see the evolution over time of the close price of the NASDAQ market.

In [ ]:
ax = nasdaq_df.plot(y=["Close"])

We can also plot the returns for all the markets.

In [ ]:
nasdaq_df.plot(y=["mom","NYSE","GSPC","DJI","RUT"], subplots=True, figsize=(20,15))

We can see that there are some common trends.

One example is the big oscillation we can observe between 2011 and 2012

---

Another interesting aspect to look into is the comparison of Nasdaq return among the different years.

As for different years we may have a different number of data points, here we compare only the years that have the same amount of data points and for which the amount corresponds to the maximum.

In [ ]:
groups = nasdaq_df["mom"].groupby(pd.Grouper(freq='A'))

max_size = 0
for _, group in groups:
    if group.values.shape[0] > max_size:
        max_size = group.values.shape[0]

years = pd.DataFrame()
for name, group in groups:
    if group.values.shape[0] == max_size:
        years[name.year] = group.values

years.plot(subplots=True, xlabel="Day of the year", title="Nasdaq return (mom) compared among different years")

Apart from some common trends between 2013 and 2014, there are no major similarities between different years.

This gives us an intuition that return depends more on other features rather than time of the year.

---

Here we report also a whisker plot for the return of NASDAQ over the years.

This plot draws a box around the 25th and 75th percentiles of the data that captures the middle 50% of observations. A line is drawn at the 50th percentile (the median) and whiskers are drawn above and below the box to summarize the general extents of the observations. Dots are drawn for outliers outside the whiskers or extents of the data.

In [ ]:
years.boxplot(rot=35)

Another interesting plot to visualize is the distribution of the featrues, we can look at this by plotting an histogram for each feature.

In [ ]:
#nasdaq_df.hist(figsize=(30,35), bins=50)

We see that a great part of the features is distributed as a gaussian centered in 0.

---

## 4 - Splitting the data: training, validation and test

As already mentioned, our dataset represents a time series. Therefore, random or stratified train/test split strategies cannot be used as they would not enable the machine learning algorithm to benefit from the fact that the data are temporally correlated with each other.

Given this considerations, the only possibility we have is to split the dataset in the most traditional way:

*   The first 60% of the entries will form the **training set**
*   The subsequent 20% will form the **validation set**
*   The last 20% will form the **test set**

Before performing the split we also compute the target vector, for each entry, the target is 1 if there is an increase in the Close price in the following entry and 0 otherwise.

$$ Target_{t} = \begin{cases} 1, & \mbox{if } Close_{t+1}>Close_{t} \\ 0, & \mbox{else} \end{cases}$$

In [ ]:
target_df =  nasdaq_df["Close"].shift(-1) - nasdaq_df["Close"]
target = np.where(target_df.values > 0, 1, 0)
target = target[:-1] # We have to remove the last value as we have no target for it

target.shape

In [ ]:
nasdaq_df = nasdaq_df.iloc[:-1,:] # We have to remove the last value as we have no target for it

nasdaq_train = nasdaq_df[:int(nasdaq_df.shape[0]*0.6)]
nasdaq_train_target = target[:int(target.shape[0]*0.6)]

nasdaq_val = nasdaq_df[int(nasdaq_df.shape[0]*0.6):int(nasdaq_df.shape[0]*0.8)]
nasdaq_val_target = target[int(target.shape[0]*0.6):int(target.shape[0]*0.8)]

nasdaq_test =  nasdaq_df[int(nasdaq_df.shape[0]*0.8):]
nasdaq_test_target = target[int(target.shape[0]*0.8):]

In [ ]:
print(nasdaq_train.shape, nasdaq_train_target.shape,
      nasdaq_val.shape, nasdaq_val_target.shape,
      nasdaq_test.shape, nasdaq_test_target.shape)

The sets are still in the form of a time series. We will procede to make it suitable for a supervised learning classical task in [this section](#transforming_the_dataset)

Given the nature of the data it is very difficult to split the dataset properly. This is because the data in the time series is closely correlated with each other and with time. Certain features refer to timesteps that go back even 200 days in the data (like EMA_200).

Therefore it is impossible to ensure that the training, validation and test set will be independent between each other. Our evaluations will always be a bit biased. Note that this is caused by the fact that the data we are analysing is a multivariate time series and by the fact that some features refer to previous timesteps.

To ensure the least interdependence between the sets we split them in the shown way. Only after this split we will proceed to build the superviesed learning dataset. Therefore for each set the first $n-1$ targets (where $n$ is the number of training days considered for the prediction) will not be used, as those are the targets that come from input that is partially of other sets or is not available.

---

Furthermore, we can look at the proportion of the target values in each set:

In [ ]:
def subset_proportions(subset):
    props = {}
    for value in set(subset):
        data_value = [i for i in subset if i==value]
        props[value] = len(data_value) / len(subset)
    return props

   
compare_props = pd.DataFrame({
    "Overall": subset_proportions(target),
    "Train": subset_proportions(nasdaq_train_target),
    "Validation": subset_proportions(nasdaq_val_target),
    "Test" : subset_proportions(nasdaq_test_target),
})

compare_props

We can see that they are pretty balanced. Each class is well represented.

All the three sets of data mantain the proportions of each class similar to the original proportions in the full dataset.

## 5 - Correlations

Now that we have split the dataset we can use our training and validation sets to gain some insights on the correlation of the features.

Let's use nasdaq_train_val for this objective.

In [ ]:
nasdaq_train_val = nasdaq_df[:int(nasdaq_df.shape[0]*0.8)]
nasdaq_train_val_target = target[:int(target.shape[0]*0.8)]

nasdaq_train_val.shape, nasdaq_train_val_target.shape

By doing this we ensure no data leakage between the training/validation sets and test set. The results on the test set are thus a fair evaluation the algorithm's performance.

---

### Autocorrelation

As we said our dataset contains a time series.

Therefore we can analyse the time series of various features.

Let's build the series for:

*   Close, the close price of the Nasdaq market
*   mom, Nasdaq return

In [ ]:
close_series = pd.Series(nasdaq_train_val["Close"])

return_series = pd.Series(nasdaq_train_val["mom"].bfill()) # We temporarly backward fill the only NaN in the mom column as we need all the values for plotting

Now that we have mapped the two series to the pandas Series object we can look at their autocorrelation plots.

The autocorrelation plot is a way of measuring and explaining the internal association between observations in a time series. We can check how strong an internal correlation is in an given amount of time.

The autocorrelation plot shows how much the value of the time series is correlated with itself $n$ timesteps in the past. $n$ is called lag.

The dotted lines are just above and below the first quartile, or within the 95% confidence interval. This will indicate the significance of the correlation. If the line is above or below the dotted line, not in between, we can say that the correlation is significant, and that the close value is correlated to time.

In [ ]:
from pandas.plotting import autocorrelation_plot

autocorrelation_plot(close_series)

In this plot we can see that the close price is highly correlated with itself in the first part of the graph. The correlation decreases with the increase of the lag as we can expect.
This is intuitive as the close price is a variable that we expect to be correlated with itself for small lags.

There is an interesting negative correlation in the second half of the plot. This could suggest a trend of stock markets to rebalance themselves every three-four years. This could also be an effect of the econimic crisis.

This suggestes that there is an association between time and close price.

In [ ]:
autocorrelation_plot(return_series)

This plot shows that the return of NASDAQ is not correlated with itself in the past and therefore we will rely on other features to predict our target.

---

Now let's look at the correlation among the features.

First we will plot the correlation matrix among features of the same category.

In [ ]:
primitive_indices = ["Close","mom"]

These are the features that resemble our target, therefore we will see the correlation of these with all the other features grouped by cathegory

### Technical indicators

Let's see the correlation between primitive features and technincal indicators.

In [ ]:
technical_indeces.remove("mom")
correlation_matrix = nasdaq_train_val[primitive_indices + technical_indeces].corr()
correlation_matrix

In [ ]:
correlation_matrix["mom"].sort_values(ascending=False)

In [ ]:
correlation_matrix["Close"].sort_values(ascending=False)

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["Close", "mom","mom1", "ROC_5", "ROC_10", "EMA_10","EMA_20"]
scatter_matrix(nasdaq_train_val[attributes])

We see that the close price is very correlated with all the EMA features and the return closely correlates with the ROC features.

### Economic indicators

In [ ]:
correlation_matrix = nasdaq_train_val[primitive_indices + economic_indices].corr()
correlation_matrix

In [ ]:
correlation_matrix["mom"].sort_values(ascending=False)

In [ ]:
correlation_matrix["Close"].sort_values(ascending=False)

In [ ]:
attributes = ["Close", "mom", "TE6", "DE4", "DBAA"]
scatter_matrix(nasdaq_train_val[attributes])

We can see many otther significant correlations in the data

### Word indices

In [ ]:
correlation_matrix = nasdaq_train_val[primitive_indices + word_indices].corr()
correlation_matrix

In [ ]:
correlation_matrix["mom"].sort_values(ascending=False)

In [ ]:
correlation_matrix["Close"].sort_values(ascending=False)

In [ ]:
attributes = primitive_indices + word_indices[:4]
scatter_matrix(nasdaq_train_val[attributes])

Here we can se that all the returns from the stock markets are very closely correlated with each other.

### All features

Now we look at the correlation between the primitive features and all other features.

In [ ]:
correlation_matrix = nasdaq_train_val.corr()
correlation_matrix

In [ ]:
correlation_matrix["mom"].sort_values(ascending=False)

In [ ]:
correlation_matrix["Close"].sort_values(ascending=False)

As we can see the return is very correlated with the returns of other stock markets and with its and their futures. We can also see a significant negative correlation with the relative change in the U.S dollar index.

The close price is very correlated with the exponential moving average and negatively correlated with economic features.

## 6 - Data transformations

Now that we have explored the attributes and their correlation we can transform our data to prepare it for the machine learning algorithms.

<a name="missing_values"></a>

### Missing values

In our dataset there are some missing values. Therefore we need to define a strategy to treat them.

Given the fact that we are working with time series we use the interpolation function provided by pandas, which interpolates the missing values using the time index.

Then to fill the remaining missing values in the first rows of the dataset, we perform backwards filling using pandas' `bfill()` function. This function propagates backwards the first valid value to fill the remaining missing values.

In [ ]:
nasdaq_train = nasdaq_train.interpolate(method='time')
nasdaq_train = nasdaq_train.bfill()

nasdaq_train_val = nasdaq_train_val.interpolate(method='time')
nasdaq_train_val = nasdaq_train_val.bfill()

nasdaq_val = nasdaq_val.interpolate(method='time')
nasdaq_val = nasdaq_val.bfill()

nasdaq_test = nasdaq_test.interpolate(method='time')
nasdaq_test = nasdaq_test.bfill()

<a name="cathegorical_values"></a>

### Categorical features

In the dataset we have just one categorical feature which is "day_of_week"

It is already expressed as an int, with the following mapping:

*   0 -> Monday
*   1 -> Tuesday
*   2 -> Wednesday
*   3 -> Thursday
*   4 -> Friday

In this case we do not need to map it to a one-hot encoding, as we want the alogrithm to benefit from the similarity given from this encoding of the days of the week.

Note that we never have data on Saturdays and Sundays, as in those days the stock market is closed.




### Feature scaling

Finally, we want to standardise our features as they have various different ranges.

To do this we perform standard scaling of the features.

In [ ]:
from sklearn.preprocessing import StandardScaler

nasdaq_train = pd.DataFrame(data=StandardScaler().fit_transform(nasdaq_train), columns=nasdaq_train.columns, index=nasdaq_train.index)

nasdaq_train_val = pd.DataFrame(data=StandardScaler().fit_transform(nasdaq_train_val), columns=nasdaq_train_val.columns, index=nasdaq_train_val.index)

nasdaq_val = pd.DataFrame(data=StandardScaler().fit_transform(nasdaq_val), columns=nasdaq_val.columns, index=nasdaq_val.index)

nasdaq_test = pd.DataFrame(data=StandardScaler().fit_transform(nasdaq_test), columns=nasdaq_test.columns, index=nasdaq_test.index)

<a name="transforming_the_dataset"></a>

### Dynamic window, transfrming the dataset for supervised learning

Now that we have prepared our data, we can build the dataset fro our prediction task.

To do this we will make use of the following function:

In [ ]:
# from https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/

def series_to_supervised(data, n_in, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (n).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in - 1, -1, -1):
        cols.append(df.shift(i))
        if i != 0:
            names += [f'{feature}(t-{i})' for feature in nasdaq_df.columns]
        else:
            names += [f'{feature}(t)' for feature in nasdaq_df.columns]

    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

This function takes as input the time series dataset and outputs a DataFrame containing for each row $n * 83$ values. These values are the values of the features for all the $n$ previous training days.

As mentioned before, for each dataset we drop the first $n-1$ lines as they would have inputs for which we don't have all the data. This is done by setting the *dropnan* flag.

---

Now we select the number of previous training days that we want to consider for our prediction.

With use the value of $60$ as it is the one chosen by the authors of the paper.

In [ ]:
n = 60

Now let's use the function described above to obtain our new dataset.

In [ ]:
nasdaq_train_y = nasdaq_train_target[n-1:]  # We drop the first n-1 targets as we do not have the data to predict them
nasdaq_train_X = series_to_supervised(nasdaq_train.values, n)

nasdaq_train_X

As we can see now our dataset is suited for the classification task.

Let's reshape it into a 3 dimensional array.

In [ ]:
nasdaq_train_X = nasdaq_train_X.values
nasdaq_train_X_3D = nasdaq_train_X.reshape(-1, n, nasdaq_train.shape[1])

nasdaq_train_X_3D.shape, nasdaq_train_y.shape

We obtain a training set with 1130 entries.

Each entry has $n * 83$ features, where n represents the number of previous training days considered and 83 is the number of features we have for each day.

---

<a name="train_val"></a>

We also convert the set containing both the training and validation set.

This set will be bigger than the union of the converted training and validation sets. This is because when we convert it we do not need to throw away the first $n-1$ entries of the validation set.

We will use this set for [time series cross validation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html)

In [ ]:
nasdaq_train_val_y = nasdaq_train_val_target[n-1:]
nasdaq_train_val_X = series_to_supervised(nasdaq_train_val.values, n)
nasdaq_train_val_X = nasdaq_train_val_X.values
nasdaq_train_val_X_3D = nasdaq_train_val_X.reshape(-1, n, nasdaq_train_val.shape[1])

nasdaq_train_val_X_3D.shape, nasdaq_train_val_y.shape


Let's convert also the validation and testing datasets.

In [ ]:
nasdaq_val_y = nasdaq_val_target[n-1:]
nasdaq_val_X = series_to_supervised(nasdaq_val.values, n)
nasdaq_val_X = nasdaq_val_X.values
nasdaq_val_X_3D = nasdaq_val_X.reshape(-1, n, nasdaq_val.shape[1])

nasdaq_val_X_3D.shape, nasdaq_val_y.shape

In [ ]:
nasdaq_test_y = nasdaq_test_target[n-1:]
nasdaq_test_X = series_to_supervised(nasdaq_test.values, n)
nasdaq_test_X = nasdaq_test_X.values
nasdaq_test_X_3D = nasdaq_test_X.reshape(-1, n, nasdaq_test.shape[1])

nasdaq_test_X_3D.shape, nasdaq_test_y.shape

# Machine learning algorithms

Now we will apply some algorithms for supervised learning and, in particular, binary classification.

## Metrics

We will evaluate our models on the following metrics:


* *Accuracy* represents the proportion of correct classifications our model makes over all the samples. In order to maximise accuracy, you'll need to maximise on the number of true positives and true negatives.
$$
ACC = \frac{TP + TN}{TP + TN + FP + FN}
$$
 
* *Precision* measures how reliable or trustworthy your classifier is. It tells you how often when the classifier predicts that the stock market close price will increase in the next day (class $1$) it actually increases. It relies on the number of $TP$'s and $FP$'s:
$$
P = \frac{TP}{TP+FP}
$$

* *Recall* measures the coverage of your classifier. It tells you how many of the actual increases your classifier can detect at all. It relies on the number of $TP$'s and $FN$'s:
$$
R = \frac{TP}{TP+FN}
$$

* Finally, $F_1$*-score* combines the two measures above to give you an overall idea of your classifier's performance. $F_1$*-score* is estimated as follows:
$$
F_1 = 2 \times \frac{P \times R}{P+R}
$$


In the paper the authors select Macro-Averaged-$F_{1}$-Measure as their evaluation metric. This is equal to the unweighted mean of the $F_1$ scores between the two classes.

Their choice is guided by the fact that accuracy may be biased towards models that tend to predict the more frequent class.

We agree with this statement.

For this project we will use all the metrics above in their *weighted* version.
*Weighted* is  similar to macro, with the difference that the resulting metric is a weighted avarage of the metric for the two classes.

In particular the one we are more interested in is the Weighted-$F_1$-Score


To see how our model perform, we will also look into:

*   the *Confusion matrix*, that represents the number of $TP, TN, FP, FN$ in a matrix:

|                  | predicted class 0.  | predicted class 1.   |
|  -------------   |   :-------------:   |    :-------------:   |
| **actual class 0** | TN                  |   FP                 |
| **actual class 1** | FN                  |   TP                 |

*  the Receiver Operating Characteristic (ROC) (when possible to plot)



To do this let's import the metrics utilities of sk-learn:

In [ ]:
from sklearn.metrics import *

We also save the *Accuracy* and *Weighted-$F_1$-Score* for each model and for both validation and testing.

This will be used at the end to plot the performance of the models and compare them.

In [ ]:
val_performance_acc = {}
val_performance_f1_weighted = {}

test_performance_acc = {}
test_performance_f1_weighted = {}

## Logistic regression

First let's train a logistic regressor.

The selected starting learning rate and the learning rate schedule have been chosen after a process of tuning by looking at the results given by the metrics on the validation set.

The starting learning rate is **0.4**

The learning rate schedule is **adaptive** ([see](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html))




In [ ]:
from sklearn.linear_model import SGDClassifier

log_reg = SGDClassifier(max_iter=400, tol=None, random_state=42,
                        loss="log", eta0=0.4, learning_rate="adaptive", penalty=None, n_jobs=-1)

log_reg.fit(nasdaq_train_X, nasdaq_train_y)

Let's see what results it achieved on the training set.

In [ ]:
pred_train = log_reg.predict(nasdaq_train_X)

confusion_matrix(nasdaq_train_y, pred_train)

Cool! It was able to find a separation boundary that correctly classifies all the training instances.


<a name="cross_val"></a>

### Cross-validation

Now let's evaluate how well it generalises performing cross-validation.

Given the fact that our input is a time series, we cannot perform stratified K-fold or normal K-fold cross-validation, instead we can just split the time series incrementally (because test indieces must always be higher than training indices [see](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html)).

The default number of splits is 5 and we will keep that.

For cross-validation we use the set containing both trianing and validation samples ([see](#train_val))

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, cross_validate

tscv = TimeSeriesSplit()
ts_split = tscv.split(nasdaq_train_val_X, nasdaq_train_val_y)

for i, [train_index, test_index] in enumerate(ts_split):
    print(f"GROUP N°{i+1}: NUM OF SAMPLES IN THE SETS-> TRAIN: {len(train_index)}, TEST: {len(test_index)}")

Now we perform cross-validation and we tell sklearn what metrics we want to evaluate



In [ ]:
metrics = ["accuracy","precision_weighted","recall_weighted","f1_weighted"]

In [ ]:
cross_val_result = cross_validate(log_reg, nasdaq_train_val_X, nasdaq_train_val_y, cv=tscv.split(nasdaq_train_val_X, nasdaq_train_val_y), scoring=metrics, n_jobs=-1)

Now let's compute the avarege value for the metrics across the 5 groups:

In [ ]:
for i in cross_val_result:
    cross_val_result[i] = cross_val_result[i].mean()
cross_val_result

We see that the result is not that good.

This is because this type of cross-validation is not as effective as stratified K-fold validation and because logistic regression may not be the best model for our task.

### Validation

Let's try evaluating the model on the validation set:

In [ ]:
pred_val = log_reg.predict(nasdaq_val_X)

In [ ]:
print(classification_report(nasdaq_val_y, pred_val, digits=4))

The lines of the ouput we are interested in are the one about accuracy and the last one containing the weighted metrics.

As we can see the result of this validation is better than the one we had for cross-validation. This is because cross-validation is not highly reliable for time series.

Let's also look at the confusion matrix:

In [ ]:
confusion_matrix(nasdaq_val_y, pred_val)

And finally we plot also the ROC curve for our model.

To do this we calculate the classification scores for the instances of the validation set.

Note that we cannot use `cross_val_predict()` as it is not compatible with TimeSeries split for obvious reasons.

In [ ]:
pred_val_scores = log_reg.decision_function(nasdaq_val_X)

In [ ]:
fpr_log_reg, tpr_log_reg, thresholds = roc_curve(nasdaq_val_y, pred_val_scores)

def plot_roc_curve(fpr, tpr, labels=None):
    for fpr_i, tpr_i, label_i in zip(fpr, tpr, labels):
        plt.plot(fpr_i, tpr_i, linewidth=2, label=label_i)
    plt.plot([0, 1], [0, 1], "k--", label="Random classifier")
    plt.axis([0, 1, 0, 1.01])
    plt.xlabel("False positive rate (fpr)")
    plt.ylabel("True positive rate (tpr)")
    plt.legend(loc="lower right")
    
plot_roc_curve([fpr_log_reg], [tpr_log_reg], labels=["Logistic regression"])

Looking at the area underneath the curve we can see how well our model performs compared to the random classifier:

In [ ]:
roc_auc_score(nasdaq_val_y, pred_val_scores)

The more the area is grater than 0.5, the more our model performs better than random.

### Considerations

This model achieves a Weighted-$F_1$-Score of 0.5723 on the validation set.

The authors of the paper run their algorithm multiple times for comparison porpuses and obtain, for 2D-CNNPred, a Macro-Averaged-$F_{1}$-Measure mean of 0.4779, with a maximum value of 0.5219 on the NASDAQ dataset.

However their runs refer to the test set, for which we still don't know how this model will preform.

It is also to be kept in mind that 2D-CNNPred is trained using all the datasets, while our model considers only NASDAQ Composite.

We will not yet look at the performance of our models on the test set.
This wil be done only at the very end of the project after we select the best performing model based on the validation performances.

In [ ]:
val_performance_acc['Log Reg'] = accuracy_score(nasdaq_val_y, pred_val)
val_performance_f1_weighted['Log Reg'] = f1_score(nasdaq_val_y, pred_val, average='weighted')

pred_test =  log_reg.predict(nasdaq_test_X)

test_performance_acc['Log Reg'] = accuracy_score(nasdaq_test_y, pred_test)
test_performance_f1_weighted['Log Reg'] = f1_score(nasdaq_test_y, pred_test, average='weighted')

## Ensemble hard voting

Now we try to use an ensemble of 4 classifiers, the used classifiers are:

*   Gaussian Naive Bayes (we infer the prior of the classes directly from the data)
*   Logistic Regression (the one that we just used)
*   Perceptron
*   Support Vector Machine

We use hard voting as the SGD classifier used to model the perceptron does not provide probability estimates.

In [ ]:
from sklearn.ensemble import VotingClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB


gnb_clf = GaussianNB()
log_clf = SGDClassifier(max_iter=400, tol=None, random_state=42,
                        loss="log", eta0=0.4, learning_rate="adaptive", penalty=None, n_jobs=-1)
perc_clf = SGDClassifier(max_iter=400, tol=None, random_state=42,
                   loss="perceptron", eta0=1, learning_rate="constant", penalty=None, n_jobs=-1)
svm_clf = SVC(random_state=42, probability=True)

voting_clf = VotingClassifier(
    estimators=[('gnb', gnb_clf), ('log', log_clf), ('perc', perc_clf), ('svm',svm_clf)],
    voting='hard')

Let's now see the evaluation metrics on each single classifier and on the voting classifier.

In [ ]:
for clf in (gnb_clf, log_clf, perc_clf, svm_clf, voting_clf):
    clf.fit(nasdaq_train_X, nasdaq_train_y)
    pred_val = clf.predict(nasdaq_val_X)
    print(clf.__class__.__name__, '\n', classification_report(nasdaq_val_y, pred_val, digits=3))

We can see that singularly SVM and Naive Bayes do nat perform very good, but in the ensemble with the Logistic Regresssion and the Perceptron (which we see performing slightly better) they provide surprising results.

Although it is hard to train the models in a way that they provide uncorrelated errors, the result we obtain with this method is an improvement with respect to the Logistic Regression classifier (treated in the previous section) alone.

The voting classifier achieves perfect results on the training set.

In [ ]:
pred_train = voting_clf.predict(nasdaq_train_X)

confusion_matrix(nasdaq_train_y, pred_train)

### Cross-validation

Now let's evaluate how well it generalises performing cross-validation (details on this have been given in [this section](#cross_val)).

In [ ]:
cross_val_result = cross_validate(voting_clf, nasdaq_train_val_X, nasdaq_train_val_y, cv=tscv.split(nasdaq_train_val_X, nasdaq_train_val_y), scoring=metrics, n_jobs=-1)

Now let's compute the avarege value for the metrics across the 5 groups:

In [ ]:
for i in cross_val_result:
    cross_val_result[i] = cross_val_result[i].mean()
cross_val_result

We see that the result is not that good.

This is because this type of cross-validation is not as effective as stratified K-fold validation.

### Validation

Let's try evaluating the model on the validation set:

In [ ]:
pred_val = voting_clf.predict(nasdaq_val_X)

In [ ]:
print(classification_report(nasdaq_val_y, pred_val, digits=4))

The lines of the ouput we are interested in are the one about accuracy and the last one containing the weighted metrics.

As we can see the result of this validation is better than the one we had for cross-validation. This is because cross-validation is not highly reliable for time series.

Let's also look at the confusion matrix:

In [ ]:
confusion_matrix(nasdaq_val_y, pred_val)

We cannot loo at the ROC Curve as the classifier uses hard voting and therefore doesn't output the probability of the predictions.

### Considerations

This model achieves a Weighted-$F_1$-Score of 0.592 on the validation set, which is better than the result achieved by Logistic Regression alone.

We will not yet look at the performance of our models on the test set.
This wil be done only at the very end of the project after we select the best performing model based on the validation performances.

In [ ]:
val_performance_acc['Voting Clf'] = accuracy_score(nasdaq_val_y, pred_val)
val_performance_f1_weighted['Voting Clf'] = f1_score(nasdaq_val_y, pred_val, average='weighted')

pred_test =  voting_clf.predict(nasdaq_test_X)

test_performance_acc['Voting Clf'] = accuracy_score(nasdaq_test_y, pred_test)
test_performance_f1_weighted['Voting Clf'] = f1_score(nasdaq_test_y, pred_test, average='weighted')

## Neural networks

This is the function we will use to compule and fit our neuarl networks.

It uses early stopping, which means that the training will stop as soon as the validation loss starts to increase. The patience for early stopping is chosen by the user.

The optimizer used is Adam which has been chosen after accurate review of available optimizers.

The function gives to the user the possibility to add a dimension to the data (that is used for 3D CNNs).

Finally the evaluation metric is the accuracy.


In [ ]:
MAX_EPOCHS = 20

def compile_and_fit(model, patience=2, add_dim=False, use_sparse=False, batch_size=32, optimizer=tf.keras.optimizers.Adam()):

    tf.keras.backend.clear_session()

    if add_dim:
        train_set = nasdaq_train_X_3D[:,:,:,np.newaxis]
        val_set = nasdaq_val_X_3D[:,:,:,np.newaxis]
    else:
        train_set = nasdaq_train_X_3D
        val_set = nasdaq_val_X_3D


    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='auto')


    loss = tf.losses.SparseCategoricalCrossentropy() if use_sparse else tf.losses.BinaryCrossentropy()

    model.compile(loss=loss,
                optimizer=optimizer,
                metrics=['accuracy'])

    history = model.fit(train_set, nasdaq_train_y, 
                        epochs=MAX_EPOCHS,
                        validation_data=(val_set, nasdaq_val_y),
                        callbacks=[early_stopping],
                        batch_size=batch_size)
        
    return history

## LSTM

Long-Short-Term memory cells are able to learn what to remember and what to forget from previously seen data.

For this reason it seems reasonable to apply them to time series prediction. As they will be able to model the correlations in the time series and predict the next step.

The LSTM layer is fed a $n*83$ vector, with $n$ the number of days.

It analyses incrementally day by day and provides one output.

In our scenario we want to output just the prediction at the last time step so we set `return_sequences` to `False`.

The output in this case is a vector containing the probabilities of classes 0 and 1. Therefore the loss function used will be `SparseCategoricalCrossentropy`.

 



### Hyperparams search

For this model we perform hyperparameter grid search using the *hparams* plugin of tensorboard.

A first pass of hyperparameters selection has shown that SGD was the best optimizer for this model.

In the second pass, reported in this section, we are investigating the number of LSTM units in the LSTM layer and the learning rate for SGD.

The chosen interval to investigate for the learning rate is (0.01, 0.1) as values in this range have been foung to provide the highest performance.

First we set the hyperparameters in which we are interested and their domains:

In [ ]:
!rm -rf ./logs

HP_NUM_UNITS_LSTM = hp.HParam('num_units_lstm', hp.IntInterval(1,200))
HP_LEARNING_RATE = hp.HParam('alpha', hp.RealInterval(0.01, 0.1))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS_LSTM, HP_LEARNING_RATE],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

Then we define a function for training the model that is parametric in the hyperparameters and interacts with tensorboard through two callbacks.

This function is very similar to the genear puprpose `compile_and_fit` function defined above, but contains the network layers definition inside andalso performs evaluation outputting the accuracy achieved on the validation set.

In [ ]:
def auto_compile_and_fit(hparams, hparam_dir, patience=2, add_dim=False, use_sparse=True, batch_size=32):

    tf.keras.backend.clear_session()

    lstm_model = tf.keras.models.Sequential([                                  
        tf.keras.layers.LSTM(hparams[HP_NUM_UNITS_LSTM], return_sequences=False),
        tf.keras.layers.Dense(units=2, activation='softmax')
    ])

    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=hparam_dir, histogram_freq=1)


    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                patience=patience,
                                                mode='auto')

    loss = tf.losses.SparseCategoricalCrossentropy() if use_sparse else tf.losses.BinaryCrossentropy()

    lstm_model.compile(loss=loss,
                optimizer=tf.keras.optimizers.SGD(learning_rate=hparams[HP_LEARNING_RATE]),
                metrics=['accuracy'])

    history = lstm_model.fit(nasdaq_train_X_3D, nasdaq_train_y, 
                        epochs=MAX_EPOCHS,
                        validation_data=(nasdaq_val_X_3D, nasdaq_val_y),
                        callbacks=[tensorboard_callback, # log metrics
                                    hp.KerasCallback(hparam_dir, hparams),
                                    early_stopping],
                        batch_size=batch_size)
    
    _, accuracy = lstm_model.evaluate(nasdaq_val_X_3D, nasdaq_val_y)
        
    return accuracy

Then, for each hyperparameter that we are interested in, we perform the grid search. Writing the resulting accuracy in the logs file.

In [ ]:
session_num = 0

for learning_rate in np.arange(HP_LEARNING_RATE.domain.min_value, HP_LEARNING_RATE.domain.max_value, 0.02):
    for num_units_lstm in np.arange(HP_NUM_UNITS_LSTM.domain.min_value, HP_NUM_UNITS_LSTM.domain.max_value, 30):
        hparams = {
            HP_LEARNING_RATE: learning_rate,
            HP_NUM_UNITS_LSTM: num_units_lstm
        }
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run_dir = 'logs/hparam_tuning/' + run_name

        with tf.summary.create_file_writer(run_dir).as_default():
            accuracy = auto_compile_and_fit(hparams, hparam_dir=run_dir)
            tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
        session_num += 1

After the search is finished, we are able to visualize in tensorboard the hyperparameters that provide the best results.

In our case these are:

*   61 LSTM cells
*   0.08 as learning rate for SGD

In [ ]:
%tensorboard --logdir logs/hparam_tuning/

### Selected model

Now we train the model with the chosen hyperparameters:

In [ ]:
lstm_model = tf.keras.models.Sequential([                                  
    # Shape [batch, days, features] => [batch, lstm_units]
    tf.keras.layers.LSTM(61, return_sequences=False),
    tf.keras.layers.Dense(units=2, activation='softmax')
])

In [ ]:
history = compile_and_fit(lstm_model, use_sparse=True, optimizer=tf.keras.optimizers.SGD(learning_rate=0.08))

As the dataset is not very big, the performance of the model depends a lot on the random initialisation of the weights.

Therefore to have a stable model I will load the one that achieved the best results.

If you want to use the model that we just trained just comment the next cell.

In [ ]:
lstm_model = keras.models.load_model('/content/drive/My Drive/Stock_market/Models/LSTM')

Let's now evaluate this model:

In [ ]:
score = lstm_model.evaluate(nasdaq_val_X_3D, nasdaq_val_y)
score

Here are the metrics on the validation set:

In [ ]:
pred_val = tf.argmax(lstm_model.predict(nasdaq_val_X_3D), axis=1)
print(classification_report(nasdaq_val_y, pred_val, digits=4))

The confusion matrix:

In [ ]:
confusion_matrix(nasdaq_val_y, pred_val)

The ROC curve:

In [ ]:
pred_val_probs = np.max(lstm_model.predict(nasdaq_val_X_3D), axis=1)
fpr_lstm, tpr_lstm, thresholds = roc_curve(nasdaq_val_y, pred_val_probs)

    
plot_roc_curve([fpr_lstm, fpr_log_reg], [tpr_lstm, tpr_log_reg], labels=["LSTM", "Logistic regression"])

Looking at the area underneath the curve we can see how well our model performs compared to the random classifier:

In [ ]:
roc_auc_score(nasdaq_val_y, pred_val_probs)

We report also the graphical representation of the network:

In [ ]:
tf.keras.utils.plot_model(lstm_model, show_shapes=True)

And the plot of the training and validation loss and accuracy (this is the history of the model just trained, as the history of the loaded model is not available):

In [ ]:
def plot_history(history):
    pd.DataFrame(history.history).plot()
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.gca().set_xlabel("Epoch")
    plt.show()

plot_history(history)

Finally, we add its performance to the dictionary that contains all the performances of our models.

In [ ]:
val_performance_acc['LSTM'] = accuracy_score(nasdaq_val_y, pred_val)
val_performance_f1_weighted['LSTM'] = f1_score(nasdaq_val_y, pred_val, average='weighted')

pred_test = tf.argmax(lstm_model.predict(nasdaq_test_X_3D), axis=1)

test_performance_acc['LSTM'] = accuracy_score(nasdaq_test_y, pred_test)
test_performance_f1_weighted['LSTM'] = f1_score(nasdaq_test_y, pred_test, average='weighted')

## 2D-CNNPred

In this section we try to recreate the 2D-CNNPred model from the paepr.

Given the great amount of information that is missing in the paper, siome parameters have been chosen by me.

To measure the loss we use `BinaryCrossEntropy()` and we use early stopping on the validation loss with a patience of 4.

Here is the architecture of the network:

In [ ]:
cnn2d_pred_model = keras.models.Sequential([
    keras.layers.Conv2D(filters=8, kernel_size=(1,83), input_shape=[60, 83, 1], activation='relu'),
    keras.layers.Dropout(rate=0.1),
    keras.layers.Conv2D(filters=8, kernel_size=(3,1), activation='relu'),
    keras.layers.Dropout(rate=0.1),
    keras.layers.MaxPooling2D(pool_size=(2,1)),
    keras.layers.Conv2D(filters=8, kernel_size=(3,1), activation='relu'),
    keras.layers.Dropout(rate=0.1),
    keras.layers.MaxPooling2D(pool_size=(2,1)),
    keras.layers.Flatten(),
    keras.layers.Dense(units=1, activation='sigmoid')
])

Let's train the model:

In [ ]:
history = compile_and_fit(cnn2d_pred_model, add_dim=True, batch_size=128, patience=4)

Now we evaluate the model on the validation set:

In [ ]:
score = cnn2d_pred_model.evaluate(nasdaq_val_X_3D[:,:,:,np.newaxis], nasdaq_val_y)
score

In [ ]:
pred_val = np.where(cnn2d_pred_model.predict(nasdaq_val_X_3D[:,:,:,np.newaxis]) > 0.5, 1, 0)
print(classification_report(nasdaq_val_y, pred_val, digits=4))

The confusion matrix:

In [ ]:
confusion_matrix(nasdaq_val_y, pred_val)

The ROC curve:

In [ ]:
pred_val_probs = cnn2d_pred_model.predict(nasdaq_val_X_3D[:,:,:,np.newaxis])
fpr_2d_cnnpred, tpr_2d_cnnpred, thresholds = roc_curve(nasdaq_val_y, pred_val_probs)

    
plot_roc_curve([fpr_lstm, fpr_log_reg, fpr_2d_cnnpred], [tpr_lstm, tpr_log_reg, tpr_2d_cnnpred], labels=["LSTM", "Logistic regression","2D-CNNPred"])

Looking at the area underneath the curve we can see how well our model performs compared to the random classifier:

In [ ]:
roc_auc_score(nasdaq_val_y, pred_val_probs)

It seems to be way worse compared to the other models, but remember, we still haven't looked at the test set.

Let's see a graphical visualization of the network:

In [ ]:
tf.keras.utils.plot_model(cnn2d_pred_model, show_shapes=True)

And a plot of the loss and accuracy:

In [ ]:
plot_history(history)

Finally, we add its performance to the dictionary that contains all the performances of our models.

In [ ]:
val_performance_acc['2D-CNNPred'] = accuracy_score(nasdaq_val_y, pred_val)
val_performance_f1_weighted['2D-CNNPred'] = f1_score(nasdaq_val_y, pred_val, average='weighted')

pred_test =  np.where(cnn2d_pred_model.predict(nasdaq_test_X_3D[:,:,:,np.newaxis]) > 0.5, 1, 0)

test_performance_acc['2D-CNNPred'] = accuracy_score(nasdaq_test_y, pred_test)
test_performance_f1_weighted['2D-CNNPred'] = f1_score(nasdaq_test_y, pred_test, average='weighted')

## MLP

### Hyperparameters search

In [ ]:
!rm -rf ./logs

HP_NUM_UNITS_HIDDEN1 = hp.HParam('num_units_hidden1', hp.IntInterval(1,200))
HP_NUM_UNITS_HIDDEN2 = hp.HParam('num_units_hidden2', hp.IntInterval(1,200))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0., 0.8))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS_HIDDEN1, HP_NUM_UNITS_HIDDEN2, HP_DROPOUT],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

Then we define a function for training the model that is parametric in the hyperparameters and interacts with tensorboard through two callbacks.

This function is very similar to the genear puprpose `compile_and_fit` function defined above, but contains the network layers definition inside andalso performs evaluation outputting the accuracy achieved on the validation set.

In [ ]:
def auto_compile_and_fit(hparams, hparam_dir, patience=2, add_dim=False, use_sparse=True, batch_size=32):

    tf.keras.backend.clear_session()

    mlp_model = keras.models.Sequential([
        keras.layers.Flatten(),
        keras.layers.Dense(hparams[HP_NUM_UNITS_HIDDEN1], activation='tanh'),
        tf.keras.layers.Dropout(rate=hparams[HP_DROPOUT]),
        keras.layers.Dense(hparams[HP_NUM_UNITS_HIDDEN2], activation='tanh'),
        tf.keras.layers.Dropout(rate=hparams[HP_DROPOUT]),
        keras.layers.Dense(units=2, activation='softmax')
    ])

    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=hparam_dir, histogram_freq=1)


    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                patience=patience,
                                                mode='auto')

    loss = tf.losses.SparseCategoricalCrossentropy() if use_sparse else tf.losses.BinaryCrossentropy()

    mlp_model.compile(loss=loss,
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.8),
                metrics=['accuracy'])

    history = mlp_model.fit(nasdaq_train_X_3D, nasdaq_train_y, 
                        epochs=MAX_EPOCHS,
                        validation_data=(nasdaq_val_X_3D, nasdaq_val_y),
                        callbacks=[tensorboard_callback, # log metrics
                                    hp.KerasCallback(hparam_dir, hparams),
                                    early_stopping],
                        batch_size=batch_size)
    
    _, accuracy = mlp_model.evaluate(nasdaq_val_X_3D, nasdaq_val_y)
        
    return accuracy

Then, for each hyperparameter that we are interested in, we perform the grid search. Writing the resulting accuracy in the logs file.

In [ ]:
session_num = 0

for num_units_hidden1 in  np.arange(HP_NUM_UNITS_HIDDEN1.domain.min_value, HP_NUM_UNITS_HIDDEN1.domain.max_value, 30):
    for num_units_hidden2 in np.arange(HP_NUM_UNITS_HIDDEN2.domain.min_value, HP_NUM_UNITS_HIDDEN2.domain.max_value, 30):
        for dropout in np.arange(HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value, 0.4):
            hparams = {
                HP_NUM_UNITS_HIDDEN1: num_units_hidden1,
                HP_NUM_UNITS_HIDDEN2: num_units_hidden2,
                HP_DROPOUT: dropout
            }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run_dir = 'logs/hparam_tuning/' + run_name

            with tf.summary.create_file_writer(run_dir).as_default():
                accuracy = auto_compile_and_fit(hparams, hparam_dir=run_dir)
                tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
            session_num += 1

In [ ]:
!kill 3813

In [ ]:
%tensorboard --logdir logs/hparam_tuning/

### Selected model

In [ ]:
mlp_model = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(units=181, activation='tanh'),
    tf.keras.layers.Dropout(rate=0.4),
    keras.layers.Dense(units=61, activation='tanh'),
    tf.keras.layers.Dropout(rate=0.4),
    keras.layers.Dense(units=2, activation='softmax')
])

In [ ]:
history = compile_and_fit(mlp_model, use_sparse=True, optimizer=keras.optimizers.SGD(learning_rate=0.08))

In [ ]:
mlp_model = tf.keras.models.load_model('/content/drive/My Drive/Stock_market/Models/MLP')

In [ ]:
pred_val = tf.argmax(mlp_model.predict(nasdaq_val_X_3D), axis=1)
print(classification_report(nasdaq_val_y, pred_val, digits=4))

In [ ]:
val_performance_acc['MLP'] = accuracy_score(nasdaq_val_y, pred_val)
val_performance_f1_weighted['MLP'] = f1_score(nasdaq_val_y, pred_val, average='weighted')

pred_test = tf.argmax(mlp_model.predict(nasdaq_test_X_3D), axis=1)

test_performance_acc['MLP'] = accuracy_score(nasdaq_test_y, pred_test)
test_performance_f1_weighted['MLP'] = f1_score(nasdaq_test_y, pred_test, average='weighted')

## Other classifiers

We have also tried a selection of various classifiers which have been less succesfull.

Here we briefly report them:

### Bagging classifier

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier

#use bootstrap=False for pasting
bag_clf = BaggingClassifier(
    SGDClassifier(max_iter=300, tol=None, random_state=42,
                        loss="log", eta0=0.4, learning_rate="adaptive", penalty=None, n_jobs=-1),
    n_estimators=200,
    max_samples=nasdaq_train_X.shape[0], max_features=0.8, 
    bootstrap=True, bootstrap_features=False,
    n_jobs=-1, random_state=42)

bag_clf.fit(nasdaq_train_X, nasdaq_train_y)

In [ ]:
pred_val = bag_clf.predict(nasdaq_val_X)
print(classification_report(nasdaq_val_y, pred_val, digits=3))

### Random forrest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=300, max_leaf_nodes=40, 
                                 n_jobs=-1, random_state=42)

rnd_clf.fit(nasdaq_train_X, nasdaq_train_y)

In [ ]:
pred_val = rnd_clf.predict(nasdaq_val_X)
print(classification_report(nasdaq_val_y, pred_val, digits=3))

## Comparison

### Validation

In [ ]:
x = np.arange(len(val_performance_acc))
width = 0.2
val_acc = list(val_performance_acc.values())


plt.ylabel('Accuracy')
plt.xlabel('Model')
plt.bar(x, val_acc, width, label='Validation')

plt.xticks(ticks=x, labels=val_performance_acc.keys(),
           rotation=45)
plt.ylim(0, np.max(val_acc) + 0.05)

plt.legend()

In [ ]:
x = np.arange(len(val_performance_f1_weighted))
width = 0.2
val_f1 = list(val_performance_f1_weighted.values())


plt.ylabel('$F_1$ Weighted')
plt.xlabel('Model')
plt.bar(x, val_f1, width, label='Validation')

plt.xticks(ticks=x, labels=val_performance_acc.keys(),
           rotation=45)
plt.ylim(0, np.max(val_f1) + 0.05)
plt.legend()

### Test

In [ ]:
x = np.arange(len(test_performance_acc))
width = 0.2

val_acc = list(val_performance_acc.values())
test_acc = list(test_performance_acc.values())

plt.bar(x - 0.17, val_acc, width, label='Validation')
plt.bar(x + 0.17, test_acc, width, label='Test')

plt.ylabel('Accuracy')
plt.xlabel('Model')

plt.xticks(ticks=x, labels=test_performance_acc.keys(),
           rotation=45)
plt.ylim(0, np.max([val_acc, test_acc]) + 0.05)

plt.legend()

In [ ]:
x = np.arange(len(test_performance_f1_weighted))
width = 0.2

val_f1 = list(val_performance_f1_weighted.values())
test_f1 = list(test_performance_f1_weighted.values())

plt.bar(x - 0.17, val_f1, width, label='Validation')
plt.bar(x + 0.17, test_f1, width, label='Test')

plt.ylabel('$F_1$ Weighted')
plt.xlabel('Model')

plt.xticks(ticks=x, labels=test_performance_f1_weighted.keys(),
           rotation=45)
plt.ylim(0, np.max([val_f1, test_f1]) + 0.05)

plt.legend()

# Visualisation and dimensionality reduction

### PCA

In [ ]:
from sklearn import decomposition

X = nasdaq_train_X

pca = sklearn.decomposition.PCA()
pca_result = pca.fit_transform(X)

# Find the predictions from the PCA model, using just the first component
print(pca.mean_.shape, pca.components_.shape, pca_result.shape)

p1,p2 = pca_result[:,0], pca_result[:,1]

fig,ax = plt.subplots(figsize=(14,11))
ax.scatter(p1, p2, alpha=.2)

for lvl in set(nasdaq_train_y):
    i = (nasdaq_train_y == lvl)
    plt.scatter(p1[i], p2[i], label=lvl)
plt.legend(loc='upper left', bbox_to_anchor=(1.1, 1))


In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(1)
plt.clf()
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)

p1,p2, p3 = pca_result[:,0], pca_result[:,1], pca_result[:,2]

plt.cla()
for lvl in set(nasdaq_train_y):
    i = (nasdaq_train_y == lvl)
    ax.scatter(p1[i], p2[i], p3[i], label=lvl)
    ax.legend()


### t-SNE

In [ ]:
from sklearn import manifold

tsne = sklearn.manifold.TSNE(n_components=2, verbose=1)
tsne_results = tsne.fit_transform(X)

In [ ]:
p1,p2 = tsne_results[:,0], tsne_results[:,1]

fig,ax = plt.subplots(figsize=(14,11))
ax.scatter(p1, p2, alpha=.2)
ax.set_aspect('equal')
for lvl in set(nasdaq_train_y):
    i = (nasdaq_train_y == lvl)
    plt.scatter(p1[i], p2[i], label=lvl)
plt.legend(loc='upper left', bbox_to_anchor=(1.1, 1))

plt.show()

In [ ]:
tsne = sklearn.manifold.TSNE(n_components=2, perplexity=5)
tsne_results = tsne.fit_transform(X)

p1,p2 = tsne_results[:,0], tsne_results[:,1]

fig,ax = plt.subplots(figsize=(14,11))
ax.scatter(p1, p2, alpha=.2)
ax.set_aspect('equal')
for lvl in set(nasdaq_train_y):
    i = (nasdaq_train_y == lvl)
    plt.scatter(p1[i], p2[i], label=lvl)
plt.legend(loc='upper left', bbox_to_anchor=(1.1, 1))

plt.show()

In [ ]:
tsne = sklearn.manifold.TSNE(n_components=2, perplexity=20)
tsne_results = tsne.fit_transform(X)

p1,p2 = tsne_results[:,0], tsne_results[:,1]

fig,ax = plt.subplots(figsize=(14,11))
ax.scatter(p1, p2, alpha=.2)
ax.set_aspect('equal')
for lvl in set(nasdaq_train_y):
    i = (nasdaq_train_y == lvl)
    plt.scatter(p1[i], p2[i], label=lvl)
plt.legend(loc='upper left', bbox_to_anchor=(1.1, 1))

plt.show()

In [ ]:
tsne = sklearn.manifold.TSNE(n_components=2, perplexity=30)
tsne_results = tsne.fit_transform(X)

p1,p2 = tsne_results[:,0], tsne_results[:,1]

fig,ax = plt.subplots(figsize=(14,11))
ax.scatter(p1, p2, alpha=.2)
ax.set_aspect('equal')
for lvl in set(nasdaq_train_y):
    i = (nasdaq_train_y == lvl)
    plt.scatter(p1[i], p2[i], label=lvl)
plt.legend(loc='upper left', bbox_to_anchor=(1.1, 1))

plt.show()

In [ ]:
tsne = sklearn.manifold.TSNE(n_components=2, perplexity=40)
tsne_results = tsne.fit_transform(X)

p1,p2 = tsne_results[:,0], tsne_results[:,1]

fig,ax = plt.subplots(figsize=(14,11))
ax.scatter(p1, p2, alpha=.2)
ax.set_aspect('equal')
for lvl in set(nasdaq_train_y):
    i = (nasdaq_train_y == lvl)
    plt.scatter(p1[i], p2[i], label=lvl)
plt.legend(loc='upper left', bbox_to_anchor=(1.1, 1))

plt.show()

In [ ]:
tsne = sklearn.manifold.TSNE(n_components=2, perplexity=50)
tsne_results = tsne.fit_transform(X)

p1,p2 = tsne_results[:,0], tsne_results[:,1]

fig,ax = plt.subplots(figsize=(14,11))
ax.scatter(p1, p2, alpha=.2)
ax.set_aspect('equal')
for lvl in set(nasdaq_train_y):
    i = (nasdaq_train_y == lvl)
    plt.scatter(p1[i], p2[i], label=lvl)
plt.legend(loc='upper left', bbox_to_anchor=(1.1, 1))

plt.show()